In [4]:
# LaGrange interpolation
# inputs: n nodes: x_i and n values: y_j, interval bounds: a,b
import math
import numpy as np
from autograd import grad
from numpy import linalg as LA
import matplotlib.pyplot as plt
from functools import reduce


def LaGrange(X, Y):
    L = np.poly1d([0])
    for k in range(0, len(X)):
        p = np.poly1d([1])
        for i in range(0, len(X)):
            if not i == k:
                p = np.polymul(p, np.poly1d([1 / (X[k] - X[i]), -X[i] / (X[k] - X[i])]))
        L = np.polyadd(L, np.polymul(Y[k], p))
    return(L)


print(np.product([5,5]))


25


In [5]:

# A is the x values for which we want to plot.
def interpolate(X, Y, start, end, iters):
    L = LaGrange(X, Y)
    A = equidist(start, end, iters)
    vals = []
    i = 0
    while i < iters:
        vals.append((L(A[i])))
        i += 1
    return vals


def LaGrange2(X, Y, x):
    n = len(X)
    L = []
    terms = []
    for k in range(0, n):
        factors = []
        for i in range(0, n):
            if (X[i] != X[k]):
                factors.append((x-X[i])/(X[k]-X[i]))
        product = np.product(factors)
        L.append(product)
        terms.append(L[k]*Y[k])

    p = math.fsum(terms)
    return p


def piecewiseLaGrange(a, b, K, x, n, f):
    i = a
    X = []
    p = np.inf
    while i < b:
        X.append([i, i+(b-a)/K])
        i += (b-a)/K
    #print(X)
    for y in X:
        #print(y)
        if x < y[1]:
            p = LaGrange2(equidist(y[0], y[1], n), func_values(f, equidist(y[0], y[1], n)), x)
            #print(p)
            break
    return p


# A is the x values for which we want to plot.
def interpolate3(start, end, iters, K, n, f):
    A = equidist(start, end, iters)
    vals = []
    i = 0
    while i < iters:
        vals.append(piecewiseLaGrange(start, end, K, A[i], n, f))
        i += 1
    return vals

# a,b interval bounds, n number of nodes
def equidist(a, b, n):
    X = []
    for k in range(0,n):
        X.append(a+k*((b-a)/n))
    return X


def chebyshev(a, b, n):
    X = []
    for k in range(1, n+1):
        X.append(1/2*(a+b)+1/2*(b-a)*math.cos((2*k-1)/(2*n)*math.pi))
    return X


def runge(X):
    Y = []
    for x in X:
        Y.append( 1/(x*x + 1) )
    return Y


# A is the x values for which we want to plot.
def interpolate2(X, Y, start, end, iters):
    A = equidist(start, end, iters)
    vals = []
    i = 0
    while i < iters:
        vals.append(LaGrange2(X, Y, A[i]))
        i += 1
    return vals


def approxInfNorm(fx, p_n):
    max = np.linalg.norm(abs(fx - p_n), np.inf)
    return max


def approx2norm(fx, p_n):
    norm = np.linalg.norm(abs(fx-p_n), 2)
    return norm


def func_values(f, X):
    fx = []
    for x in X:
        fx.append(f(x))
    fx = np.asarray(fx)
    return fx


def F(x):
    return math.cos(2 * math.pi * x)


def G(x):
    return math.exp(3*x)*math.sin(2*x)



In [6]:
def error_vs_n(N, norm):
    i = 1
    list = []
    theoretic = []
    while i < N:
        X = equidist(0, 1, 100 * i)
        fx = func_values(F, X)
        interpol_vals = func_values(F, chebyshev(0, 1, i))
        p_n = interpolate2(chebyshev(0, 1, i), interpol_vals, 0, 1, 100 * i)
        list.append( norm( fx, p_n ) )
        theoretic.append( (2*math.pi)**(i+1)/(math.factorial(i+1)) )
        i += 1
    return list, theoretic


def expError(N, norm):
    i = 1
    list = []
    while i < N:
        X = equidist(0, math.pi/4, 100*i)
        fx = func_values(G, X)
        interpol_vals = func_values(G, chebyshev(0, math.pi/4, i))
        p_n = interpolate2(chebyshev(0,1, i), interpol_vals, 0, 1, 100*i)
        list.append(norm(fx, p_n))
        i += 1
    return list


def expError2(K, norm, n):
    i = 1
    list = []
    X = equidist(0, math.pi / 4, 100*n)
    while i < K:
        fx = func_values(G, X)
        p_n = interpolate3(0, math.pi/4, 100*n, i, n, G)
        list.append(norm(fx, p_n))
        i += 1
    return list


#lol = expError2(1000, approxInfNorm, 3)
#plt.semilogy(lol, "r")
#plt.show()

#errors, theoretical = error_vs_n(40, approxInfNorm)
#errors2, theoretical2 = error_vs_n(40, approx2norm)

#G_errors = expError(40, approxInfNorm)
#G_errors2 = expError(40, approx2norm)

#plt.semilogy(G_errors, "r")
#plt.semilogy(G_errors2, "b")
#plt.show()

#plt.semilogy(errors, "r")
#plt.semilogy(errors2, "b")
#plt.semilogy(theoretical)
#plt.show()


In [7]:

def costfunction(f, a, b, X):
    N = 1000
    CX = 0
    A = equidist(a, b, N)
    fmerket = [val for (key, val) in f]
    for x in A:
        p_n_x = LaGrange2(X, func_values(F, X), x)
        CX = CX + ( f[x]-p_n_x )**2
    CX = CX*((b-a)/N)
    return CX


A = equidist(0, 1, 1000)
def interpolcreate(X):
    p_n = []
    f = []
    for x in A:
        f.append(F(x))
        p_n.append(LaGrange2(X, func_values(F, X), x))

    return p_n, f


X = chebyshev(0, 1, 7)
a = 0
b = 1
n = 7
p_n, f = interpolcreate(chebyshev(0, 1, n))

p_n = np.array(p_n)
#f = np.array(f)


from functools import partial #Denne gjør mye morsomt

#Det første du må gjøre er å skrive om costfunction til costfunction(f, a, b, X)

#Antatt at man har verdier for f, start a og slutt b
f, a, b = f, 0, 1 #Bytt gjerne disse til de riktige verdiene

differentiableCost = partial(costfunction, f, a, b)
costDerived = grad(differentiableCost)

print(costDerived(X)) #Prøver å printe gradienten med en X-verdi



# The cost function takes as input X - the interpolation nodes,
# returning C(X) -- the cost of interpolating with those nodes.
def cost(X):
    CX = 0
    i = 0
    while i < 1000:
        CX = CX + ((b-a)/1000)*((f[i]-p_n[i])**2)
        i = i + 1
    return CX


def terms():
    terms = []
    for i in range(0, 1000):
        terms.append(((f[i]-p_n[i])**2))
    return terms


terms = terms()


def LaGrange3(X, Y, x):
    n = len(X)
    L = []
    terms = []
    for k in range(0, n):
        factors = []
        for i in range(0, n):
            if (X[i] != X[k]):
                factors.append((x-X[i])/(X[k]-X[i]))
        product = np.product(factors)
        L.append(product)
        terms.append(L[k]*Y[k])

    p = math.fsum(terms)
    return p


def cost3(X, f):
    n = len(X)
    L = []
    terms = []
    for k in range(0, n):
        factors = []
        for i in range(0, n):
            if (X[i] != X[k]):
                factors.append((x - X[i]) / (X[k] - X[i]))
        product = np.product(factors)
        L.append(product)
        terms.append(L[k] * f(X[k]))
    p = math.fsum(terms)


def cost2(X):
    return ((b-a)/1000)*np.sum(terms)


#print(grad(cost)(0.34))

#dc = grad(costfunction)
#DC = grad(cost)
#DC2 = grad(cost2)
#print(DC)
#print(DC2)
#print(dc)


cost = cost(chebyshev(0, 1, n))

TypeError: cannot unpack non-iterable float object